# Multi-Agent Collaboration 
---

### What is Multi-Agent Collaboration?
Multi-Agent Collaboration refers to the process where multiple autonomous agents—each capable of independent decision-making—work together to achieve common or complementary objectives. This concept is widely used in fields like artificial intelligence, robotics, distributed computing, and simulation, and it involves several key aspects:

- **Effective Communication and Coordination**:
Agents exchange information and align their actions to collectively achieve a goal, ensuring that tasks are organized and synchronized.

- **Autonomous, Distributed Decision-Making**:
Each agent operates independently, making local decisions while contributing to a broader strategy, which enhances flexibility and fault tolerance.

- **Adaptive Task Specialization**:
Agents focus on specific roles or subtasks based on their capabilities, and they adjust their strategies through iterative feedback, leading to improved overall performance.


### Key Advantages
- **Efficiency Through Task Specialization**:
By assigning specific roles to different agents, the system can handle complex tasks in parallel. This specialization allows each agent to focus on its area of expertise, resulting in faster and more effective problem-solving.

- **Scalability and Flexibility**:
AutoGen's structured communication and dynamic task allocation enable the system to scale easily. It can adapt to varying project complexities by adding or reassigning agents as needed, ensuring that the collaboration remains robust even as demands change.

- **Enhanced Iterative Refinement**:
The framework’s built-in feedback loops and iterative dialogue facilitate continuous improvement. Agents can refine their outputs based on real-time feedback, leading to more accurate and cohesive final results.

**Reference**
- [AutoGen paper: Enabling Next-Gen LLM Applications via Multi-Agent Conversation](https://arxiv.org/abs/2308.08155)
- [Multi-Agent Collabration Concept](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#network) 

In [2]:
from dotenv import load_dotenv
import os
import json

load_dotenv("../../.env")

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = (
    os.getenv("AZURE_OPENAI_API_KEY", "")
    if len(os.getenv("AZURE_OPENAI_API_KEY", "")) > 0
    else None
)
azure_openai_chat_deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_embedding_deployment_name = os.getenv(
    "AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME", "text-embedding-ada-002"
)
openai_api_version = os.getenv("OPENAI_API_VERSION", "2024-06-01")

In [3]:
autogen_aoai_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=azure_openai_endpoint,
    model=azure_openai_chat_deployment_name,
    api_version=openai_api_version,
    api_key=azure_openai_key,
)

<br>


## 🧪 Step 1. Define the Agentic Architecture
- Before building the agentic pipeline, we need to design the message, topic, agent and message routing logic. 
- You should define the terminate condition for the pipeline.

### Message, Topic, Agent Definition

```markdown
```python

# Message Definition
ChatMessage
termination = TextMentionTermination("TERMINATE")

# Agent Definition
travel_info_agent = AssistantAgent(gpt-4o)
local_agent  = AssistantAgent(gpt-4o)
language_agent  = AssistantAgent(gpt-4o)
travel_summary_agent  = AssistantAgent(gpt-4o)


# Message Routing Definition
RoundRobinGroupChat
```
```

### Define Agents

In the next section we will define the agents that will be used in the travel planning team.

In [ ]:
travel_info_agent = AssistantAgent(
    "planner_agent",
    model_client=autogen_aoai_client,
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request.",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=autogen_aoai_client,
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided.",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=autogen_aoai_client,
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=AzureOpenAIChatCompletionClient(model="gpt-4o"),
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed final travel plan. You must ensure that the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.",
)

<br>

## 🧪 Step 2. Execute the Agents TeamChat

### Execute the group chat with the termination condition

In [ ]:
termination = TextMentionTermination("TERMINATE")
group_chat = RoundRobinGroupChat(
    [travel_info_agent, local_agent, language_agent, travel_summary_agent],
    termination_condition=termination,
)
await Console(group_chat.run_stream(task="Plan a 3 day trip to Nepal."))

---------- user ----------
Plan a 3 day trip to Nepal.


---------- planner_agent ----------
Certainly! Here’s a 3-day itinerary for your trip to Nepal, focusing on Kathmandu and its surroundings, which offers a blend of culture, history, and natural beauty.

### Day 1: Explore Kathmandu

**Morning:**
- **Pashupatinath Temple:** Start your day at this UNESCO World Heritage Site, which is one of the holiest Hindu temples dedicated to Lord Shiva. Arrive early to witness the morning rituals and ceremonies by the river.
  
**Lunch:**
- **Thamel Area:** Head over to the bustling Thamel district for lunch. Try some local dishes such as dal bhat or momos at a traditional restaurant.

**Afternoon:**
- **Boudhanath Stupa:** After lunch, visit one of the largest stupas in the world. It is a significant pilgrimage site for Buddhists and offers a serene atmosphere. Don’t forget to circumambulate the stupa and explore the nearby monasteries.

**Evening:**
- **Thamel Shopping:** Return to Thamel for some shopping. Look for souvenirs, handicrafts, and trek

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Plan a 3 day trip to Nepal.', type='TextMessage'), TextMessage(source='planner_agent', models_usage=RequestUsage(prompt_tokens=40, completion_tokens=754), content="Certainly! Here’s a 3-day itinerary for your trip to Nepal, focusing on Kathmandu and its surroundings, which offers a blend of culture, history, and natural beauty.\n\n### Day 1: Explore Kathmandu\n\n**Morning:**\n- **Pashupatinath Temple:** Start your day at this UNESCO World Heritage Site, which is one of the holiest Hindu temples dedicated to Lord Shiva. Arrive early to witness the morning rituals and ceremonies by the river.\n  \n**Lunch:**\n- **Thamel Area:** Head over to the bustling Thamel district for lunch. Try some local dishes such as dal bhat or momos at a traditional restaurant.\n\n**Afternoon:**\n- **Boudhanath Stupa:** After lunch, visit one of the largest stupas in the world. It is a significant pilgrimage site for Buddhists and offe